In [1]:
import os
import sys
sys.path.append('/home/amine.mcharrak/clear2025/dglearn/')

from dglearn import *
from causal_helpers import load_adjacency_matrix
from connectome_cyclic_data_generator import sample_cyclic_data

###########################################################################################

# example graph structure with 6 variables
#n_vars = 6
#edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 1), (5, 4)]
#var_names = {i:"$X_%d$"%(i+1) for i in range(n_vars)}

# generate random parameters for true graph structure
#B_support = edges2array(edges, n_vars) - np.eye(n_vars)
#B_sampled, s_sampled = sample_param_unif(B_support, B_low=0.2, B_high=0.8, var_low=1.0, var_high=3.0)

# generate samples from distribution specified by sampled parameters
#X = sample_graph(B_sampled, s_sampled, 10000)

###########################################################################################

# Specify the folder where the cyclic connectome data was saved
folder_cyclic = '/home/amine.mcharrak/connectome/data/'
B_true_weighted = load_adjacency_matrix(os.path.join(folder_cyclic, 'A_init_t_ordered_adj_matrix_with_cycles.npy'))
X, B_sampled = sample_cyclic_data(B_true_weighted, n_samples=10000, noise_type='gaussian')
B_support = (B_sampled != 0).astype(int)

###########################################################################################

# learn structure using tabu search, plot learned structure
tabu_length = X.shape[1] # number of variables
patience = 4

manager = CyclicManager(X, bic_coef=0.5)
learned_support, best_score, log = tabu_search(manager, tabu_length, patience, first_ascent=False, verbose=1) # returns a binary matrix as learned support

Adjacency matrix loaded from /home/amine.mcharrak/connectome/data/A_init_t_ordered_adj_matrix_with_cycles.npy
Add edge: 203 -> 267, Delta Score: 2.638e-01 best score: 0.26375689618730663
Add edge: 143 -> 151, Delta Score: 2.381e-01 best score: 0.5018704670002684
Add edge: 169 -> 163, Delta Score: 2.366e-01 best score: 0.7384375448608145
Add edge: 139 -> 95, Delta Score: 2.356e-01 best score: 0.9740165590396123
Add edge: 200 -> 188, Delta Score: 2.285e-01 best score: 1.2024928335402258
Add edge: 267 -> 58, Delta Score: 2.242e-01 best score: 1.4267354801166605
Add edge: 103 -> 261, Delta Score: 2.191e-01 best score: 1.6457926556222657
Add edge: 184 -> 174, Delta Score: 2.146e-01 best score: 1.8604364947168164
Add edge: 103 -> 90, Delta Score: 2.047e-01 best score: 2.0651242681017608
Add edge: 218 -> 55, Delta Score: 2.045e-01 best score: 2.269575174213352
Add edge: 200 -> 207, Delta Score: 1.996e-01 best score: 2.469213505753963
Add edge: 105 -> 103, Delta Score: 1.991e-01 best score: 2.

In [1]:
# perform virtual edge correction
print("virtual edge correction...")
learned_support = virtual_refine(manager, learned_support, patience=0, max_path_len=6, verbose=1) # still is still a binary matrix

virtual edge correction...


NameError: name 'virtual_refine' is not defined

In [ ]:

# remove any reducible edges
learned_support = reduce_support(learned_support, fill_diagonal=False)

# plot true and learned directed graphs
plot_collection({'true': edges, 'learned': array2edges(learned_support)}, n_vars, name_list=var_names,
                latex=False, n_cols=2, node_size=600, font_size=14, width=2, save_path="./assets/dg6_learning_result.png")

# compute performance metric: SHD
true_graph = AdjacencyStucture(n_vars, edge_list=edges)
search = GraphEquivalenceSearch(true_graph)
search.search_dfs()
equiv_class = [binary2array(bstr) for bstr in search.visited_graphs]

In [ ]:
display(B_support)
display(B_sampled)